# Estimación de peso y dimensiones de los envíos de Mercado Libre

# Materia: Análisis y curación de datos

## Análisis del dataset

Se pide: A partir de lo visto en la teoría de la materia y del segundo laboratorio, diagramar una comunicación en formato textual o interactivo describiendo la solución de las actividades propuestas a continuación. Al final de las mismas se proveen actividades opcionales (no obligatorias) que pueden resultar de interés.

### Actividades Propuestas:
    1.	Eliminar valores cuyo status sea `404` , luego eliminar la columna `status` del dataset ya que solo es útil para limpieza.

    2.	Eliminar los valores NaN de las columnas con prefijo `SHP_`. Estas son aquellas que representan o peso o dimensiones de un item.

    3.	Agrupar por item id y calcular mediana de peso y medidas. De esta forma debería quedar una única fila por cada item_id.

    4.	Parsear la columna de atributos y extraer a columnas propias aquellos atributos cuyo `id` sea `BRAND` o `MODEL`. Estos atributos representan marca o modelo que el vendedor del item ingresó en la publicación. [Opcional] No es necesario limitarse a estos dos atributos, se puede probar quedarse con los N atributos más frecuentes.

    5.	Transformar variables categóricas en números (Se recomienda OneHotEncoding) para las columnas (Sugerencia: arrancar con un sample de ~10K items)

        a.	`CATALOG_PRODUCT_ID`
        b.	`CONDITION`
        c.	`DOMAIN_ID`
        d.	`SELLER_ID`
        e.	`BRAND` (extraída en 4)
        f.	`MODEL`(extraída en 4)

    6.	En caso de tener alguna variable no medida (en nuestro caso `PRICE`) imputar sus valores utilizando kNN.

    7.	Medir las distribuciones de las variables como histogramas, realizar normalizaciones e identificar outliers con los métodos vistos en clase. Hacer análisis de estos outliers y considerar si sería correcto o no eliminarlos del dataset. Sugerencia: Identificar outliers de las columnas `SHP_WEIGHT` y `SHP_VOLUME`, donde `SHP_VOLUME` se define como el producto de las dimensiones.

    8.	[Opcional] Si están interesados en usar texto y técnicas de NLP sobre el título:
        a.	Realizar preprocesamiento del texto, se recomienda leer este artículo.
        b.	Para convertir el título en vectores se recomienda ver tf-idf. wiki, sklearn.

    9.	[Opcional] Aplicar PCA y decidir si existe un subespacio del espacio de features donde viven los datos y son bien descriptos. Graficar.

    10.	[Opcional] Aplicar Mixturas Gaussianas (GMM) para hacer clustering. 

La comunicación debe estar apuntada a un público técnico pero sin conocimiento del tema particular, como por ejemplo, sus compañeros de clase o stakeholders del proyecto. Idealmente, además del documento se debería generar una presentación corta para stakeholders explicando el análisis realizado sobre los datos y las conclusiones obtenidas de tal análisis.

Se evaluarán los siguientes aspectos:

    ●	El informe debe contener un mensaje claro y presentado de forma concisa.
    ●	Los gráficos deben aplicar los conceptos de percepción visual vistos en clase.
    ●	Se debe describir o estimar la significancia estadística de su trabajo.


In [39]:
import pandas as pd
import random
import matplotlib.pyplot as plt
import numpy
import seaborn
import scipy as sc

In [40]:
random.seed(0)

El dataset corresponde a un muestreo de envíos durante tres meses de 2018 en MLB (Mercado Libre Brasil), filtrados según métodos de envíos y por carriers (empresas de correos a cargo del envío propiamente dicho). A su vez, fue muestreado de forma no uniforme para romper de forma manual la distribución de los datos.
Los datos son:[meli_dataset_20190426.csv] (https://drive.google.com/open?id=1tNUKD1lf1z8C7LPpCruiDl6SKBAHb79v).

In [41]:
DATASET = 'meli_dataset_20190426.csv'

In [42]:
df_original = pd.read_csv(DATASET, low_memory=False)


In [52]:
df= df_original

In [53]:
df = df.head(10000) 

In [54]:
df.columns

Index(['ITEM_ID', 'SHP_WEIGHT', 'SHP_LENGTH', 'SHP_WIDTH', 'SHP_HEIGHT',
       'ATTRIBUTES', 'CATALOG_PRODUCT_ID', 'CONDITION', 'DOMAIN_ID', 'PRICE',
       'SELLER_ID', 'STATUS', 'TITLE'],
      dtype='object')

#### ¿Qué significa cada columna?

* ITEM_ID: id unívoco de cada item publicado. (Ofuscado)
* SHP_WEIGHT: peso del paquete informado por el correo.
* SHP_LENGTH: largo del paquete informado por el correo.
* SHP_WIDTH: ancho del paquete informado por el correo.
* SHP_HEIGHT: altura del paquete informado por el correo.
* ATTRIBUTES: atributos como marca y modelo, entre otros, en formato json-lines
* CATALOG_PRODUCT_ID: id del catálogo (ofuscado).
* CONDITION: condición de venta (nuevo o usado).
* DOMAIN_ID: id de la categoría a la que pertenece la publicación.
* PRICE: precio en reales.
* SELLER_ID: id del vendedor (ofuscado).
* STATUS: estado de la publicación (activa, cerrada, pausada, etc.)
* TITLE: título de la publicación.

In [55]:
df.sample(5)

,ITEM_ID,SHP_WEIGHT,SHP_LENGTH,SHP_WIDTH,SHP_HEIGHT,ATTRIBUTES,CATALOG_PRODUCT_ID,CONDITION,DOMAIN_ID,PRICE,SELLER_ID,STATUS,TITLE
3067,OZ1QXLGK62,NaN,NaN,NaN,NaN,"[{'id': 'BATTERY_CAPACITY', 'name': 'Capacidad...",H53U1H7Q5G,new,MLB-CELLPHONE_BATTERIES,90.99,CLB1WKKA1S,paused,Bateria Apple Iphone 6s Plus 5.5 2915 Mah - N...
9514,H7WCH3YU1G,4180.0,36.0,25.0,11.0,"[{'id': 'PART_NUMBER', 'name': 'Número da peça...",H53U1H7Q5G,new,NaN,142.90,MQICEHKRH5,closed,Kit Troca De Óleo Castrol Maxi 5w40 Vw Up 1.0 ...
3034,RSYETSVFDD,NaN,NaN,NaN,NaN,NaN,H53U1H7Q5G,NaN,NaN,NaN,YH42FTPGLU,under_review,Bravecto 10 A 20 Kg - 12x Sem Juros - Val: 12...
286,IE4KG7Q0Z3,710.0,25.0,25.0,5.0,"[{'id': 'BRAND', 'name': 'Marca', 'value_id': ...",SBJWOPHCYY,new,MLB-MALE_UNDERWEAR,137.90,Z1K0MERPN7,active,Kit 7 Cuecas Selene Boxer Modal 11010-001 P/eg...
8747,ZXQQ8R6BRH,NaN,NaN,NaN,NaN,NaN,H53U1H7Q5G,NaN,NaN,NaN,W5MKGBGVEK,under_review,Tenis Infantil Patrulha Canina Luzes Botinha M...


## Actividad 1:

Eliminar valores cuyo status sea `404` , luego eliminar la columna `status` del dataset ya que solo es útil para limpieza.


In [56]:
# df.filter(["STATUS"])!= "404"

df = df[df.STATUS != "404"]
df = df.drop(columns=['STATUS'])
df.sample(5)

,ITEM_ID,SHP_WEIGHT,SHP_LENGTH,SHP_WIDTH,SHP_HEIGHT,ATTRIBUTES,CATALOG_PRODUCT_ID,CONDITION,DOMAIN_ID,PRICE,SELLER_ID,TITLE
9926,UB7FJHWSRN,260.0,19.0,16.9,8.5,"[{'id': 'BRAND', 'name': 'Marca', 'value_id': ...",H53U1H7Q5G,new,MLB-SPORTSWEAR,129.40,O8RKTQN28E,Bermuda Venum Camo Preto Camuflado Cinza
2369,SET0K5ZH79,1770.0,32.0,31.0,18.0,"[{'id': 'BRAND', 'name': 'Marca', 'value_id': ...",H53U1H7Q5G,new,MLB-LOAFERS_AND_OXFORDS,120.00,V50K5DROU4,Kit 03 Mocassim Masculino Casual Sapatênis Fre...
3261,YLDGRR5DZN,NaN,NaN,NaN,NaN,"[{'id': 'ITEM_CONDITION', 'name': 'Condição do...",H53U1H7Q5G,new,MLB-ALTERNATIVE_THERAPY_EQUIPMENT,49.97,JXR7OI2YOO,Febreze Air Aromatizador Ambiente - Linen & Sk...
4018,SX98Y7LMJ7,1972.0,40.0,20.0,10.0,"[{'id': 'BRAND', 'name': 'Marca', 'value_id': ...",H53U1H7Q5G,new,MLB-CAR_SEAT_COVERS,123.64,TFB5TYGPX5,Capa Banco De Couro Courvin Celta 2007 Com Lo...
1451,OO6EMXXY3P,NaN,NaN,NaN,NaN,"[{'id': 'BRAND', 'name': 'Marca', 'value_id': ...",H53U1H7Q5G,new,MLB-LOAFERS_AND_OXFORDS,89.49,ILPIZK31HA,Mocassim Sapatenis Masculino Casual Kit 2 Pare...


## Actividad 2:

Eliminar los valores NaN de las columnas con prefijo `SHP_`. Estas son aquellas que representan o peso o dimensiones de un item.


In [57]:
# df = df.dropna(subset=['SHP_WEIGHT'])
# df = df.dropna(subset=['SHP_LENGTH'])
# df = df.dropna(subset=['SHP_WIDTH'])
# df = df.dropna(subset=['SHP_HEIGHT'])

# o bien:
df = df.dropna(subset=['SHP_WEIGHT', 'SHP_LENGTH', 'SHP_WIDTH', 'SHP_HEIGHT'])

#o bien

# df= df.dropna(df.filter(regex ='SHP_'))


#df.sample(3)



## Actividad 3:

Agrupar por item id y calcular mediana de peso y medidas. De esta forma debería quedar una única fila por cada item_id.

In [58]:
medianas = df.groupby('ITEM_ID').median()
medianas


,SHP_WEIGHT,SHP_LENGTH,SHP_WIDTH,SHP_HEIGHT,PRICE
ITEM_ID,,,,,
A0BKGTRNTM,775.0,50.0,20.0,10.0,750.00
A0JIHFAMOP,6100.0,70.0,25.0,5.0,119.90
A0NG260OMQ,464.0,20.0,11.0,10.0,349.90
A0WL6O455O,150.0,25.0,25.0,11.0,21.99
A1957258HH,3719.0,42.0,34.0,13.0,849.00
...,...,...,...,...,...
ZZFFUBJ43D,3880.0,36.0,24.0,13.0,145.90
ZZGF7K64VC,1609.0,22.5,20.0,18.0,185.00
ZZQD93N8HV,200.0,23.0,20.5,6.5,NaN


## Actividad 4:

Parsear la columna de atributos y extraer a columnas propias aquellos atributos cuyo `id` sea `BRAND` o `MODEL`. Estos atributos representan marca o modelo que el vendedor del item ingresó en la publicación. [Opcional] No es necesario limitarse a estos dos atributos, se puede probar quedarse con los N atributos más frecuentes.

## Actividad 5:

Transformar variables categóricas en números (Se recomienda OneHotEncoding) para las columnas (Sugerencia: arrancar con un sample de ~10K items)

## Actividad 6: 

En caso de tener alguna variable no medida (en nuestro caso `PRICE`) imputar sus valores utilizando kNN.

## Actividad 7:

Medir las distribuciones de las variables como histogramas, realizar normalizaciones e identificar outliers con los métodos vistos en clase. Hacer análisis de estos outliers y considerar si sería correcto o no eliminarlos del dataset. Sugerencia: Identificar outliers de las columnas `SHP_WEIGHT` y `SHP_VOLUME`, donde `SHP_VOLUME` se define como el producto de las dimensiones.